In [141]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
driver = webdriver.Chrome(executable_path='C:\chromedriver\chromedriver.exe')


In [142]:
##HELPERFUNCTIONS

def get_matchLinks(page):
    matchesLinks = []
    ## https://www.hltv.org/results?offset=0
    driver.get('https://www.hltv.org/results?offset='+ str(page) +'&stars=1')
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    ## FIND MATCHES
    for div in soup.findAll('div', attrs={'class': 'results'}):
        for a in div.findAll('a', attrs={'class': 'a-reset'}):
            link = a['href']
            if (link[:8] == "/matches"):
                matchesLinks.append(link)
    return matchesLinks

def get_page_data(link):
    url = 'https://www.hltv.org/' + str(link)
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)    
    return soup

def get_game_links(soup):
    statlinks = []
    for div in soup.findAll('div', attrs={'class': 'results-center-stats'}):
        for a in div.findAll('a'):
            statlinks.append(a['href'])
    return statlinks

def get_game_stats(link):
    result = get_page_data(link)
    return result

In [143]:
## GAME STAT INFO

#print(data[8][0])
#LINK /stats/matches/mapstatsid/118030/astralis-vs-og
def gather_player_stats(link):
    colnames = ["Player","K (hs)","A (f)","D","KAST","K-D Diff","ADR","FK Diff","Rating","Team"]

    #localLink = data[8][0]
    localLink = link
    result = get_game_stats(localLink)
    result = result.findAll('table', {'class':'stats-table'})
    page_source = result
    results = pd.read_html(str(result))

    MapID = localLink.split("/")[4]

    #print(MapID)
    #print(Map)
    df_team1 = pd.DataFrame(results[0])
    df_team1['Team'] = df_team1.columns[0]
    df_team1.columns = colnames
    #df_team1["Player"].replace('\r\n',regex=True,inplace=True)

    df_team2 = pd.DataFrame(results[1])
    df_team2['Team'] = df_team2.columns[0]
    df_team2.columns = colnames
    #df_team2["Player"].replace('\r\n',regex=True,inplace=True)

    game_df = df_team1.append(df_team2,ignore_index=True)
    game_df['MatchID'] = MapID
    #game_df['Map'] 
    #print(game_df)
    #game_df.to_csv(index=True)
    return game_df


In [144]:
##BASIC INFO

def gather_match_stats(links):
    match_df = pd.DataFrame()
    ##GET DATE from https://www.hltv.org/matches/2347834***
    names = ["Date","Tournament","Team1","Result1","Team2","Result2","Map","MapID","StatLinks","Players"]
    data = []
    maps = []
    mapsID = []
    players = []

    for div in soup.findAll('div', attrs={'class': 'match-page'}):
        pageDate = div.find('div', attrs={'class': 'date'})
        pageTournament = div.find('div', attrs={'class': 'event text-ellipsis'})
        data.append(pageDate.text)
        data.append(pageTournament.text)

    ##GET FINALRESULT1 from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'team1-gradient'}):
        pageTeam1 = div.find('div', attrs={'class': 'teamName'})
        pageResult1 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
        data.append(pageTeam1.text)
        data.append(pageResult1.text)

    ##GET FINALRESULT2 from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'team2-gradient'}):
        pageTeam2 = div.find('div', attrs={'class': 'teamName'})
        pageResult2 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
        data.append(pageTeam2.text)
        data.append(pageResult2.text)

    ##GET MAPS AND MAPSID from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'stats-menu-link'}):
        map1 = div.findAll('div', attrs= {'class': 'dynamic-map-name-full'})
        for div in map1:
            maps.append(div.text)
            mapsID.append(div['id'])

    #map_df = pd.DataFrame(maps)
    data.append(maps)
    #mapsID_df = pd.DataFrame(mapsID)
    data.append(mapsID)


    ##GET DETAILED STATS LINKS
    data.append(links)

    ##GET PLAYER INFO
    for div in soup.findAll('div', attrs={'class': 'lineups'}):
        for table in div.findAll('td',attrs={'class': 'player player-image'}):
            for a in table.findAll('a'):
                players.append(a['href'])
    player_df = pd.DataFrame(players)


    data_dict = {'data':data}
    data_df = pd.DataFrame(data_dict)
    #data_df.index = names
    #-----------
    #maps_df = pd.DataFrame(data_df.iloc[[7]].values)
    #print(data_df.iloc[[7]].values)
    #print(data_df)
    for i in range(1,len(maps)):
        #print(maps[i])
        #print(mapsID[i])
        #print(links[i-1])
        game_df = gather_player_stats(links[i-1])
        game_df['Map'] = maps[i]
        match_df = match_df.append(game_df)

    return match_df
    #match_df.to_clipboard()



    #data_df.iloc[7].values[0][0] -> maps
    #data_df.iloc[8].values[0][0] -> mapID
    #data_df.iloc[9].values[0][0] -> players

In [150]:
##MAIN

#CAREFUL FETCHES PAGE (runtime=1)
matchesLinks = get_matchLinks(0) #goes to results page to fetch matchLinks
#print(pd.DataFrame(matchesLinks))

for i in range(0,3): #len(matchesLinks)
    #CAREFUL FETCHES PAGE
    soup = get_page_data(matchesLinks[i]) #gets the soup for matchpage from match links
    gamelinks = get_game_links(soup) #gets the gamelinks from match links
    #CAREFUL FETCHES 2-5 pages
    print(gather_match_stats(gamelinks)) #gets stats from gamelinks

                                                     0
0    /matches/2347742/liquid-vs-dignitas-blast-prem...
1    /matches/2347741/g2-vs-endpoint-blast-premier-...
2    /matches/2347740/astralis-vs-og-blast-premier-...
3    /matches/2347395/ex-winstrike-vs-ago-esea-prem...
4    /matches/2347742/liquid-vs-dignitas-blast-prem...
..                                                 ...
98   /matches/2346925/nip-vs-faze-esl-pro-league-se...
99   /matches/2346924/vitality-vs-g2-esl-pro-league...
100  /matches/2346923/mousesports-vs-faze-esl-pro-l...
101  /matches/2346883/sinners-vs-saw-pinnacle-cup-2021
102  /matches/2346922/ence-vs-nip-esl-pro-league-se...

[103 rows x 1 columns]
118068
118074
     Player   K (hs)   A (f)   D   KAST  K-D Diff    ADR  FK Diff  Rating  \
0     EliGE   15 (9)   3 (1)  15  57.1%         0   83.4        1    1.01   
1       NAF   11 (3)   3 (0)  14  61.9%        -3   78.5        0    0.88   
2    FalleN    9 (4)   3 (1)  17  57.1%        -8   48.5        1    0.

for i in range(0,len(results)):
    print(results[i])
df = pd.DataFrame(result[0])
print(df)